In [171]:
# !jt -t chesterish -T -N -kl

In [172]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from catboost import CatBoostRegressor

from tqdm.notebook import tqdm

from pprint import pprint   


from ast import literal_eval
import json

import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler

from IPython.display import Audio
from russian_names import RussianNames

from catboost import CatBoostRegressor, Pool, cv
from huperopt_class import HyperOpt

In [173]:
RANDOM_SEED = 42

In [174]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [175]:
data = pd.read_csv('data_for_stacking.csv')
cat_features_ids = ['bodyType', 'brand', 'color','descr_labels', 'fuelType', 
                    'model_name', 'vehicleTransmission',
                    'pts', 'privod', 'wheel', 'state', 'descr_labels']

In [176]:
X = data.query('sample == 1').drop(['sample'], axis=1)
y = X['price']

X.drop('price', axis=1, inplace=True, errors='ignore')

In [177]:
VAL_SIZE=0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [178]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [179]:
data.duplicated(subset='sell_id', keep='first').value_counts()

False    117620
True      62433
dtype: int64

# XGBoost

In [180]:
data = pd.read_csv('data_for_stacking.csv')
cat_features_ids = ['bodyType', 'brand', 'color', 'fuelType', 
                    'model_name', 'vehicleTransmission',
                    'pts', 'privod', 'wheel', 'state', 'descr_labels']

for colum in cat_features_ids:
    data[colum] = data[colum].astype('category').cat.codes

In [181]:
X = data.query('sample == 1').drop(['sample'], axis=1)
y = X['price']

X.drop('price', axis=1, inplace=True, errors='ignore')
X_sub = data.query('sample == 0').drop(['sample'], axis=1)
VAL_SIZE=0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
y_train = np.log(y_train)
y_test = np.log(y_test)

In [182]:
eval_set = [(X_test, y_test)]

In [183]:
import xgboost as xgb


model_xgb = xgb.XGBRegressor(base_score=0.5, 
                             booster='gbtree',
                             colsample_bylevel=1,
                             colsample_bynode=1, 
                             colsample_bytree=1, 
                             gamma=0, #0.15340366103115533, #0,
                             importance_type='gain', 
                             learning_rate=0.08,
                             
                             max_delta_step=0,
                             max_depth=12, #6, #7, 
                             min_child_weight=1, 
                             missing=None, 
                             n_estimators=5000,
                             n_jobs=20, 
                             nthread=None, 
                             random_state=0,
                             reg_alpha=0, #1.8400184528746324, #0, 
                             reg_lambda=1, #1.0868061353806249,#1, 
                             scale_pos_weight=1, 
                             seed=RANDOM_SEED,
                             silent=None, 
                             subsample=0.75, 
                             verbosity=0, 
                             objective='reg:squarederror',
                             verbose=True
                            )


# {'gamma': 0.15340366103115533, 'l2_leaf_reg': 2.0, 'learning_rate': 0.00853890793354474, 'max_depth': 5.971733628773733, 'reg_alpha': 1.8400184528746324, 'reg_lambda': 1.0868061353806249}

In [184]:
model_xgb.fit(X_train, y_train, eval_metric=mape)

[19:20:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.08, max_delta_step=0, max_depth=12,
             min_child_weight=1, missing=None, monotone_constraints='()',
             n_estimators=5000, n_jobs=20, nthread=20, num_parallel_tree=1,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=42, silent=None, subsample=0.75, tree_method='exact',
             validate_parameters=1, verbose=True, verbosity=1)

In [185]:
predict = model_xgb.predict(X_test)

predict = np.e ** predict
y_test = np.e ** y_test

# оцениваем точность
MAPE = f'{(mape(y_test, predict))*100:0.4f}'
print(f"Точность модели по метрике MAPE: {MAPE}%")

Точность модели по метрике MAPE: 9.7719%


In [186]:
# import hyperopt 
# import numpy as np
# from huperopt_class import HyperOpt
# xgb_kwargs = {'cv':5, 'n_estimators':5000, 'verbose':False,'rounds':50,
#               'X_train':X_train, 'X_val':X_test, 'y_train':y_train, 'y_val':y_test,
#               'n_jobs':20 }

# params_space = {'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
#                 'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 1e-2),
#                 'max_depth': hyperopt.hp.uniform('max_depth', 2, 10),
#                 'gamma': hyperopt.hp.uniform('gamma', 0, 3),
#                 'reg_alpha': hyperopt.hp.uniform('reg_alpha', 0, 2),
#                 'reg_lambda': hyperopt.hp.uniform('reg_lambda', 0, 2)
#                }

# xgb_hyperopt_inst = HyperOpt(**xgb_kwargs)
# trials = hyperopt.Trials()
# best = hyperopt.fmin(
#     xgb_hyperopt_inst.hyperopt_xgb_score,
#     space=params_space,
#     algo=hyperopt.tpe.suggest,
#     max_evals=20,
#     trials=trials,
#     rstate=np.random.RandomState(42)
# )
# print(best)


# {'gamma': 0.15340366103115533, 'l2_leaf_reg': 2.0, 'learning_rate': 0.00853890793354474, 'max_depth': 5.971733628773733, 'reg_alpha': 1.8400184528746324, 'reg_lambda': 1.0868061353806249}

In [187]:
import winsound
freq = 2000 # Set frequency To 2500 Hertz
dur = 700 # Set duration To 1000 ms == 1 second
winsound.Beep(freq, dur)